# Connect to Hive

In [1]:
from pyspark.sql import SparkSession

# Add here your team number teamx
team = 19

# location of your Hive database in HDFS
# warehouse = "project/hive/warehouse"
warehouse = "/user/team19/project/warehouse"

spark = SparkSession.builder\
        .appName("{} - spark ML".format(team))\
        .master("yarn")\
        .config("hive.metastore.uris", "thrift://hadoop-02.uni.innopolis.ru:9883")\
        .config("spark.sql.warehouse.dir", warehouse)\
        .config("spark.sql.avro.compression.codec", "snappy")\
        .enableHiveSupport()\
        .getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/07 18:37:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/07 18:37:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/07 18:37:06 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/05/07 18:37:06 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
25/05/07 18:37:07 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
# pip freeze
!java -version

openjdk version "1.8.0_402"
OpenJDK Runtime Environment (build 1.8.0_402-b06)
OpenJDK 64-Bit Server VM (build 25.402-b06, mixed mode)


In [3]:
spark

# list Hive databases

In [4]:
spark.sql("SHOW DATABASES").show()

+--------------------+
|           namespace|
+--------------------+
|             default|
|             retake1|
|             root_db|
|                show|
|     team0_projectdb|
|    team11_projectdb|
|           team12_db|
|team12_hive_proje...|
|    team12_projectdb|
|    team13_projectdb|
|    team14_projectdb|
|    team15_projectdb|
|    team16_projectdb|
|    team17_projectdb|
|    team18_projectdb|
|    team19_projectdb|
|     team1_projectdb|
|    team20_projectdb|
| team21_projectdb_v2|
| team21_projectdb_v3|
+--------------------+
only showing top 20 rows



# Read hive tables

In [5]:
spark.sql("USE team19_projectdb").show()
spark.sql("SHOW TABLES").show()

++
||
++
++

+----------------+--------------------+-----------+
|       namespace|           tableName|isTemporary|
+----------------+--------------------+-----------+
|team19_projectdb|  evaluation_results|      false|
|team19_projectdb|      gbt_gridsearch|      false|
|team19_projectdb|   gbt_metrics_tuned|      false|
|team19_projectdb|      hosts_bucketed|      false|
|team19_projectdb|listings_partitioned|      false|
|team19_projectdb|       lr_gridsearch|      false|
|team19_projectdb|    lr_metrics_tuned|      false|
|team19_projectdb|  model1_predictions|      false|
|team19_projectdb|  model2_predictions|      false|
|team19_projectdb|          q1_results|      false|
|team19_projectdb|          q2_results|      false|
|team19_projectdb|          q3_results|      false|
|team19_projectdb|          q4_results|      false|
|team19_projectdb|          q5_results|      false|
|team19_projectdb|          q6_results|      false|
|team19_projectdb|review_scores_buc...|      false|

In [6]:
hosts_bucketed = spark.read.format("avro").table('team19_projectdb.hosts_bucketed')
listings_partitioned = spark.read.format("avro").table('team19_projectdb.listings_partitioned')
review_scores_bucketed = spark.read.format("avro").table('team19_projectdb.review_scores_bucketed')

In [7]:
# hosts_bucketed.show()
hosts_bucketed.columns

['host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications']

In [8]:
hosts_bucketed.show(1)

25/05/07 18:37:29 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


+----------------+--------------------+---------+----------+-------------+----------+------------------+------------------+--------------------+-------------------+-------------------------+-------------------+
|         host_id|            host_url|host_name|host_since|host_location|host_about|host_response_time|host_response_rate|host_acceptance_rate|host_listings_count|host_total_listings_count| host_verifications|
+----------------+--------------------+---------+----------+-------------+----------+------------------+------------------+--------------------+-------------------+-------------------------+-------------------+
|71592872.0000000|https://www.airbn...|      Ian|2016-05-12|         NULL|      NULL|within a few hours|       100.0000000|                NULL|          1.0000000|                1.0000000|email,phone,reviews|
+----------------+--------------------+---------+----------+-------------+----------+------------------+------------------+--------------------+------------

In [9]:
listings_partitioned.columns

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'host_id',
 'street',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'price',
 'number_of_reviews',
 'reviews_per_month',
 'city',
 'country']

In [10]:
listings_partitioned.show(1)

+--------+--------------------+--------------+------------+-------------+----------------+--------------------+-------------+------------+------------+---------+---------+---------+----------+-----------------+-----------------+-----------------+-------+
|      id|         listing_url|     scrape_id|last_scraped|         name|         host_id|              street|property_type|   room_type|accommodates|bathrooms| bedrooms|     beds|     price|number_of_reviews|reviews_per_month|             city|country|
+--------+--------------------+--------------+------------+-------------+----------------+--------------------+-------------+------------+------------+---------+---------+---------+----------+-----------------+-----------------+-----------------+-------+
|10908984|https://www.airbn...|20160706203047|  2016-07-07|Room "Shanti"|52727228.0000000|colina, Playas de...|    Apartment|Private room|   2.0000000|0.0000000|1.0000000|1.0000000|24.0000000|        3.0000000|        0.5600000|Playas 

In [11]:
review_scores_bucketed.columns

['listing_id',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value']

In [12]:
review_scores_bucketed.show(1)

+----------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+
|listing_id|review_scores_rating|review_scores_accuracy|review_scores_cleanliness|review_scores_checkin|review_scores_communication|review_scores_location|review_scores_value|
+----------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+
|  17895559|                NULL|                  NULL|                     NULL|                 NULL|                       NULL|                  NULL|               NULL|
+----------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+
only showing top 1 row



In [13]:
from pyspark.sql.functions import col
result_df = (
    listings_partitioned
    .join(
        hosts_bucketed,
        on="host_id",
        how="left"
    )
    .join(
        review_scores_bucketed,
        on=col("id") == col("listing_id"),
        how="left"
    )
)


In [14]:
result_df.columns

['host_id',
 'id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'street',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'price',
 'number_of_reviews',
 'reviews_per_month',
 'city',
 'country',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'listing_id',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value']

In [15]:
total_rows = result_df.count()
print(f"Total rows: {total_rows}")

Total rows: 247476


In [16]:
from pyspark.sql.functions import col, count, when
null_counts = result_df.agg(
    *[count(when(col(c).isNull(), c)).alias(c) for c in result_df.columns]
).show(vertical=True)

25/05/07 18:38:01 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0-----------------------------
 host_id                     | 0      
 id                          | 0      
 listing_url                 | 0      
 scrape_id                   | 0      
 last_scraped                | 0      
 name                        | 223    
 street                      | 0      
 property_type               | 4      
 room_type                   | 0      
 accommodates                | 39     
 bathrooms                   | 764    
 bedrooms                    | 301    
 beds                        | 476    
 price                       | 4076   
 number_of_reviews           | 0      
 reviews_per_month           | 60968  
 city                        | 235    
 country                     | 0      
 host_url                    | 0      
 host_name                   | 260    
 host_since                  | 259    
 host_location               | 1143   
 host_about                  | 98959  
 host_response_time          | 57987  
 host_response_rate      

In [17]:
features = ['last_scraped', 'street', 'property_type', 'room_type', 'accommodates','bathrooms', 'bedrooms', 'beds', 'number_of_reviews', 'reviews_per_month', 'city',
           'country', 'host_since', 'host_location', 'host_response_time','host_response_rate','host_listings_count', 'host_total_listings_count', 'host_verifications', 'review_scores_rating',
           'review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value']
label = 'price'

In [18]:
result_df = result_df.select(features + [label]).na.drop()
result_df = result_df.withColumnRenamed("price","label")
result_df.show()

+------------+--------------------+-------------+---------------+------------+---------+---------+---------+-----------------+-----------------+-------------+--------------+----------+--------------------+------------------+------------------+-------------------+-------------------------+--------------------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+-----------+
|last_scraped|              street|property_type|      room_type|accommodates|bathrooms| bedrooms|     beds|number_of_reviews|reviews_per_month|         city|       country|host_since|       host_location|host_response_time|host_response_rate|host_listings_count|host_total_listings_count|  host_verifications|review_scores_rating|review_scores_accuracy|review_scores_cleanliness|review_scores_checkin|review_scores_communication|review_scores_location|review_scores_value|      label|
+------------+--------------

In [19]:
result_df.describe().show(vertical=True)

-RECORD 0-----------------------------------------------------------
 summary                     | count                                
 host_id                     | 247476                               
 id                          | 247476                               
 listing_url                 | 247476                               
 scrape_id                   | 247476                               
 name                        | 247253                               
 street                      | 247476                               
 property_type               | 247472                               
 room_type                   | 247476                               
 accommodates                | 247437                               
 bathrooms                   | 246712                               
 bedrooms                    | 247175                               
 beds                        | 247000                               
 price                       | 243

In [19]:
total_rows = result_df.count()
print(f"Total rows: {total_rows}")

Total rows: 151055


In [20]:
result_df.columns

['last_scraped',
 'street',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'number_of_reviews',
 'reviews_per_month',
 'city',
 'country',
 'host_since',
 'host_location',
 'host_response_time',
 'host_response_rate',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'label']

In [21]:
from pyspark.sql.functions import to_date, year, month, dayofmonth

df_with_dates = result_df.withColumn(
    "date_parsed",
    to_date("last_scraped", format="yyyy-MM-dd")
).withColumn(
    "last_scraped_year", year("date_parsed")
).withColumn(
    "month", month("date_parsed")
).withColumn(
    "day", dayofmonth("date_parsed")
)

In [22]:
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCols
from pyspark.sql.functions import sin, cos, pi
import math

class CyclicDateEncoder(Transformer, HasInputCol, HasOutputCols):
    def __init__(self, inputCol=None, outputCols=None):
        super(CyclicDateEncoder, self).__init__()
        self._set(inputCol=inputCol, outputCols=outputCols)
        
    def _transform(self, df):
        input_col = self.getInputCol()
        output_cols = self.getOutputCols()
        if input_col == "month":
            return df.withColumn(
                output_cols[0],
                sin(2 * pi() * col(input_col) / 12)
            ).withColumn(
                output_cols[1],
                cos(2 * pi() * col(input_col) / 12)
            )
        elif input_col == "day":
            return df.withColumn(
                output_cols[0],
                sin(2 * pi() * col(input_col) / 31)
            ).withColumn(
                output_cols[1],
                cos(2 * pi() * col(input_col) / 31)
            )

In [23]:
month_encoder = CyclicDateEncoder(
    inputCol="month",
    outputCols=["last_scrapped_month_sin", "last_scrapped_month_cos"]
)
day_encoder = CyclicDateEncoder(
    inputCol="day",
    outputCols=["last_scrapped_day_sin", "last_scrapped_day_cos"]
)
df_encoded = month_encoder.transform(df_with_dates)
df_encoded = day_encoder.transform(df_encoded)

In [24]:
df_encoded.columns

['last_scraped',
 'street',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'number_of_reviews',
 'reviews_per_month',
 'city',
 'country',
 'host_since',
 'host_location',
 'host_response_time',
 'host_response_rate',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'label',
 'date_parsed',
 'last_scraped_year',
 'month',
 'day',
 'last_scrapped_month_sin',
 'last_scrapped_month_cos',
 'last_scrapped_day_sin',
 'last_scrapped_day_cos']

# Feature selection

In [25]:
df_encoded = df_encoded.drop("last_scraped", "date_parsed","day","month")
df_encoded.columns

['street',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'number_of_reviews',
 'reviews_per_month',
 'city',
 'country',
 'host_since',
 'host_location',
 'host_response_time',
 'host_response_rate',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'label',
 'last_scraped_year',
 'last_scrapped_month_sin',
 'last_scrapped_month_cos',
 'last_scrapped_day_sin',
 'last_scrapped_day_cos']

In [26]:
from pyspark.sql.functions import to_date, year, month, dayofmonth

df_with_dates = df_encoded.withColumn(
    "date_parsed",
    to_date("host_since", format="yyyy-MM-dd")
).withColumn(
    "year_host_since", year("date_parsed")
).withColumn(
    "month", month("date_parsed")
).withColumn(
    "day", dayofmonth("date_parsed")
)

In [27]:
df_with_dates.columns

['street',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'number_of_reviews',
 'reviews_per_month',
 'city',
 'country',
 'host_since',
 'host_location',
 'host_response_time',
 'host_response_rate',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'label',
 'last_scraped_year',
 'last_scrapped_month_sin',
 'last_scrapped_month_cos',
 'last_scrapped_day_sin',
 'last_scrapped_day_cos',
 'date_parsed',
 'year_host_since',
 'month',
 'day']

In [28]:
month_encoder = CyclicDateEncoder(
    inputCol="month",
    outputCols=["host_since_month_sin", "host_since_month_cos"]
)
day_encoder = CyclicDateEncoder(
    inputCol="day",
    outputCols=["host_since_day_sin", "host_since_day_cos"]
)
df_encoded = month_encoder.transform(df_with_dates)
df_encoded = day_encoder.transform(df_encoded)


In [29]:
df_encoded = df_encoded.drop("month", "day","date_parsed","host_since")
df_encoded.columns

['street',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'number_of_reviews',
 'reviews_per_month',
 'city',
 'country',
 'host_location',
 'host_response_time',
 'host_response_rate',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'label',
 'last_scraped_year',
 'last_scrapped_month_sin',
 'last_scrapped_month_cos',
 'last_scrapped_day_sin',
 'last_scrapped_day_cos',
 'year_host_since',
 'host_since_month_sin',
 'host_since_month_cos',
 'host_since_day_sin',
 'host_since_day_cos']

In [30]:
df_encoded.columns

['street',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'number_of_reviews',
 'reviews_per_month',
 'city',
 'country',
 'host_location',
 'host_response_time',
 'host_response_rate',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'label',
 'last_scraped_year',
 'last_scrapped_month_sin',
 'last_scrapped_month_cos',
 'last_scrapped_day_sin',
 'last_scrapped_day_cos',
 'year_host_since',
 'host_since_month_sin',
 'host_since_month_cos',
 'host_since_day_sin',
 'host_since_day_cos']

In [31]:
df_encoded = df_encoded.drop("last_scraped")

In [32]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, Word2Vec, Tokenizer, RegexTokenizer
from pyspark.sql.functions import col

categoricalCols = ['city','country','host_location','host_verifications', 'property_type','room_type', 'host_response_time']
textCols = ['street']
others = ['accommodates','bathrooms','bedrooms','beds','number_of_reviews','reviews_per_month',
          'host_response_rate','host_listings_count', 'host_total_listings_count','review_scores_rating','review_scores_accuracy',
          'review_scores_cleanliness', 'review_scores_checkin','review_scores_communication', 'review_scores_location', 'review_scores_value',
         'last_scraped_year','host_since_month_sin','host_since_month_cos','host_since_day_sin','host_since_day_cos','year_host_since']

In [33]:
df_encoded.select('street').show(5)

+--------------------+
|              street|
+--------------------+
|Mission District,...|
|New York, NY 1002...|
|Ficial District, ...|
|Panthéon, Paris, ...|
|Copenhagen, Capit...|
+--------------------+
only showing top 5 rows



In [34]:
# unique_count = df_encoded.select("city").distinct().count()
# print(unique_count)

In [35]:
# unique_count = df_encoded.select("host_location").distinct().count()
# print(unique_count)

In [36]:
# unique_count = df_encoded.select("host_verifications").distinct().count()
# print(unique_count)

In [37]:
# unique_count = df_encoded.select("country").distinct().count()
# print(unique_count)

In [38]:
# unique_count = df_encoded.select("street").distinct().count()
# print(unique_count)

In [39]:
from pyspark.ml.feature import RegexTokenizer, Word2Vec
from pyspark.sql.functions import col

tokenizer = RegexTokenizer(
    inputCol="street", 
    outputCol="city_tokens",
    pattern="[,\s]+"
)

In [40]:
word2Vec = Word2Vec(
    vectorSize=10,
    minCount=1,
    windowSize=5,
    inputCol="city_tokens",
    outputCol="city_vec"
)

In [41]:
indexers = [ StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c)).setHandleInvalid("skip") for c in categoricalCols ]
encoders = [ OneHotEncoder(inputCol=indexer.getOutputCol(), outputCol="{0}_encoded".format(indexer.getOutputCol())) for indexer in indexers ]
assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders] + others, outputCol= "features")

In [67]:
df_encoded.describe().show(truncate=False)

ConnectionRefusedError: [Errno 111] Connection refused

In [42]:
# You can create a pipeline to use only a single fit and transform on the data.
pipeline = Pipeline(stages=[tokenizer, word2Vec] + indexers + encoders + [assembler])


# Fit the pipeline ==> This will call the fit functions for all transformers if exist
model=pipeline.fit(df_encoded)
# Fit the pipeline ==> This will call the transform functions for all transformers
data = model.transform(df_encoded)

# data.show()

# We delete all features and keep only the features and label columns
data = data.select(["features", "label"])


from pyspark.ml.feature import VectorIndexer

# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4
# distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)
transformed = featureIndexer.transform(data)

# Display the output Spark DataFrame
# transformed.show()


25/05/07 15:13:18 WARN DAGScheduler: Broadcasting large task binary with size 1584.1 KiB


# Feature extraction

# Split the dataset

In [43]:
transformed = transformed.withColumn("label", col("label").cast("double"))

In [44]:
transformed.show(1)

25/05/07 15:14:04 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB


+--------------------+-----+--------------------+
|            features|label|     indexedFeatures|
+--------------------+-----+--------------------+
|(9436,[12,3080,31...|149.0|(9436,[12,3080,31...|
+--------------------+-----+--------------------+
only showing top 1 row



In [61]:
transformed.describe().show()

+-------+------------------+
|summary|             label|
+-------+------------------+
|  count|            151055|
|   mean|132.59360497831915|
| stddev|141.10245841580925|
|    min|               0.0|
|    max|             999.0|
+-------+------------------+



In [45]:
#  split the data into 60% training and 40% test (it is not stratified)
(train_data, test_data) = transformed.randomSplit([0.6, 0.4], seed = 10)

def run(command):
    import os
    return os.popen(command).read()

train_data.select("features", "label")\
    .coalesce(1)\
    .write\
    .mode("overwrite")\
    .format("json")\
    .save("project/data/train")

# Run it from root directory of the repository
run("hdfs dfs -cat project/data/train/*.json > ../data/train.json")

test_data.select("features", "label")\
    .coalesce(1)\
    .write\
    .mode("overwrite")\
    .format("json")\
    .save("project/data/test")

# Run it from root directory of the repository
run("hdfs dfs -cat project/data/test/*.json > ../data/test.json")

25/05/07 15:14:20 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB
25/05/07 15:14:47 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB


''

In [46]:
optimal_partitions = 12
train_data = train_data.repartition(optimal_partitions)

In [47]:
train_data.rdd.getNumPartitions()

25/05/07 15:15:12 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB


12

# First model

## Build a model

In [48]:
from pyspark.ml.regression import LinearRegression
# Create Linear Regression Model
lr = LinearRegression(maxIter=30)

# Fit the data to the pipeline stages
model_lr = lr.fit(train_data)

25/05/07 15:15:38 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
25/05/07 15:16:01 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
25/05/07 15:16:08 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
25/05/07 15:16:09 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
25/05/07 15:16:12 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/05/07 15:16:12 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
25/05/07 15:16:13 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
25/05/07 15:16:13 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
25/05/07 15:16:14 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
25/05/07 15:16:14 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
25/05/07 15:16:14 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
25/05/07 15:16:15 WARN DAGSchedul

## Predict for test data

In [49]:
predictions = model_lr.transform(test_data)
predictions.show()

25/05/07 15:17:20 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


+--------------------+-----+--------------------+------------------+
|            features|label|     indexedFeatures|        prediction|
+--------------------+-----+--------------------+------------------+
|(9436,[0,3082,309...| 56.0|(9436,[0,3082,309...| 76.44810687011432|
|(9436,[0,3082,309...| 65.0|(9436,[0,3082,309...| 93.52075228571721|
|(9436,[0,3082,309...| 99.0|(9436,[0,3082,309...|  97.8595575441559|
|(9436,[0,3082,309...| 55.0|(9436,[0,3082,309...| 69.85328822933207|
|(9436,[0,3082,309...|175.0|(9436,[0,3082,309...| 99.18242281728362|
|(9436,[0,3082,309...| 70.0|(9436,[0,3082,309...| 96.90623292131295|
|(9436,[0,3082,309...| 98.0|(9436,[0,3082,309...|107.63512722587984|
|(9436,[0,3082,309...| 80.0|(9436,[0,3082,309...|102.37314584747764|
|(9436,[0,3082,309...| 85.0|(9436,[0,3082,309...| 62.16993914863906|
|(9436,[0,3082,309...| 70.0|(9436,[0,3082,309...|101.64227380466673|
|(9436,[0,3082,309...| 85.0|(9436,[0,3082,309...| 85.63457153808577|
|(9436,[0,3082,309...| 58.0|(9436,

## Evaluate the model

In [50]:
from pyspark.ml.evaluation import RegressionEvaluator 

# Evaluate the performance of the model
evaluator1_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
evaluator1_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")

rmse = evaluator1_rmse.evaluate(predictions)
r2 = evaluator1_r2.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on test data = {}".format(rmse))
print("R^2 on test data = {}".format(r2))

25/05/07 15:17:40 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
25/05/07 15:18:01 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


Root Mean Squared Error (RMSE) on test data = 73.45921960024111
R^2 on test data = 0.7229996254387352


In [51]:
# Non fine-tuned first model
metrics_filename = "Non_fine_tuned_linear_regression_metrics"

In [52]:
metrics_data = [{"RMSE": rmse, "R2": r2, "model": "linear_reg_1"}]
metrics_df = spark.createDataFrame(metrics_data)
metrics_df.write.format("json").mode("overwrite").save(f"project/big_data_project/output/{metrics_filename}")
# model2.write().overwrite().save("project/big_data_project/models/model2")

In [53]:
run("hdfs dfs -get project/big_data_project/output/Non_fine_tuned_linear_regression_metrics ../output/model1_no_tune")

get: `../output/model1_no_tune/Non_fine_tuned_linear_regression_metrics/_SUCCESS': File exists


''

## Hyperparameter optimization

In [55]:
model_lr.params

[Param(parent='LinearRegression_c6603fbad9a8', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'),
 Param(parent='LinearRegression_c6603fbad9a8', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'),
 Param(parent='LinearRegression_c6603fbad9a8', name='epsilon', doc='The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber'),
 Param(parent='LinearRegression_c6603fbad9a8', name='featuresCol', doc='features column name.'),
 Param(parent='LinearRegression_c6603fbad9a8', name='fitIntercept', doc='whether to fit an intercept term.'),
 Param(parent='LinearRegression_c6603fbad9a8', name='labelCol', doc='label column name.'),
 Param(parent='LinearRegression_c6603fbad9a8', name='loss', doc='The loss function to be optimized. Supported options: squaredError, huber.'),
 Param(parent='LinearRegression_c6603fbad9a8', name='m

In [56]:
train_data.cache()

DataFrame[features: vector, label: double, indexedFeatures: vector]

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator 

import numpy as np
grid = ParamGridBuilder()\
    .addGrid(model_lr.regParam, np.logspace(-2, -1, 2))\
    .addGrid(lr.elasticNetParam, [0.2, 0.6])\
    .build()

cv = CrossValidator(estimator = lr, 
                    estimatorParamMaps = grid, 
                    evaluator = evaluator1_rmse,
                    parallelism = 5,
                    numFolds=3)

cvModel = cv.fit(train_data)
bestModel = cvModel.bestModel
bestModel

In [60]:
from pyspark.sql import Row
# Create a list of Rows with parameter combinations and metrics
cv_results = []
for i, param_combo in enumerate(grid):
    cv_results.append(Row(
        reg_param=float(param_combo[lr.regParam]),
        elastic_net_param=float(param_combo[lr.elasticNetParam]),
        rmse=float(cvModel.avgMetrics[i])
    ))

# Create a Spark DataFrame from the results
cv_results_df = spark.createDataFrame(cv_results)

# Show the results
print("Cross-Validation Results:")
cv_results_df.show(truncate=False)
# Optionally save the results
cv_results_df.coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("project/big_data_project/output/cv_results_lr")
run("hdfs dfs -get project/big_data_project/output/cv_results_lr ./output/model1")

Cross-Validation Results:
+---------+-----------------+-----------------+
|reg_param|elastic_net_param|rmse             |
+---------+-----------------+-----------------+
|0.01     |0.2              |75.12670885936343|
|0.01     |0.6              |75.06411943921381|
|0.1      |0.2              |74.88251784248949|
|0.1      |0.6              |74.43698490144723|
+---------+-----------------+-----------------+



## Best model 1


In [53]:
from pprint import pprint
model1 = bestModel
pprint(model1.extractParamMap())

{Param(parent='LinearRegression_a9df76969288', name='predictionCol', doc='prediction column name.'): 'prediction',
 Param(parent='LinearRegression_a9df76969288', name='regParam', doc='regularization parameter (>= 0).'): 0.1,
 Param(parent='LinearRegression_a9df76969288', name='maxIter', doc='max number of iterations (>= 0).'): 30,
 Param(parent='LinearRegression_a9df76969288', name='tol', doc='the convergence tolerance for iterative algorithms (>= 0).'): 1e-06,
 Param(parent='LinearRegression_a9df76969288', name='solver', doc='The solver algorithm for optimization. Supported options: auto, normal, l-bfgs.'): 'auto',
 Param(parent='LinearRegression_a9df76969288', name='standardization', doc='whether to standardize the training features before fitting the model.'): True,
 Param(parent='LinearRegression_a9df76969288', name='fitIntercept', doc='whether to fit an intercept term.'): True,
 Param(parent='LinearRegression_a9df76969288', name='loss', doc='The loss function to be optimized. Supp

## Save the model to HDFS

In [54]:
model1.write().overwrite().save("project/big_data_project/models/model1")

# Run it from root directory of the repository
run("hdfs dfs -get project/big_data_project/models/model1 model1")

get: `model1/model1/data/_SUCCESS': File exists                                 
get: `model1/model1/metadata/_SUCCESS': File exists
get: `model1/model1/metadata/part-00000': File exists


''

In [55]:
# run("hdfs dfs -ls project/big_data_project/models/model1")

## Predict for test data using best model1

In [56]:
predictions = model1.transform(test_data)
predictions.show()

25/05/06 09:49:21 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


+--------------------+-----+--------------------+------------------+
|            features|label|     indexedFeatures|        prediction|
+--------------------+-----+--------------------+------------------+
|(9436,[0,3082,309...| 56.0|(9436,[0,3082,309...| 74.49045917259855|
|(9436,[0,3082,309...| 65.0|(9436,[0,3082,309...| 91.17054378757439|
|(9436,[0,3082,309...| 99.0|(9436,[0,3082,309...| 97.21836977507473|
|(9436,[0,3082,309...| 55.0|(9436,[0,3082,309...| 67.71750060563363|
|(9436,[0,3082,309...|175.0|(9436,[0,3082,309...| 98.25505187191948|
|(9436,[0,3082,309...| 70.0|(9436,[0,3082,309...| 96.12909031431809|
|(9436,[0,3082,309...| 98.0|(9436,[0,3082,309...|105.55048234974811|
|(9436,[0,3082,309...| 80.0|(9436,[0,3082,309...| 101.4196470645652|
|(9436,[0,3082,309...| 85.0|(9436,[0,3082,309...| 61.58927615376342|
|(9436,[0,3082,309...| 70.0|(9436,[0,3082,309...|100.66625029541183|
|(9436,[0,3082,309...| 85.0|(9436,[0,3082,309...| 85.12105835718285|
|(9436,[0,3082,309...| 58.0|(9436,

In [57]:
predictions.select("label", "prediction")\
    .coalesce(1)\
    .write\
    .mode("overwrite")\
    .format("csv")\
    .option("sep", ",")\
    .option("header","true")\
    .save("project/big_data_project/output/model1_predictions.csv")

# Run it from root directory of the repository
run("hdfs dfs -cat project/big_data_project/output/model1_predictions.csv/*.csv > ../output/model1_predictions.csv")

25/05/06 09:49:36 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB


''

## Evaluate the best model1

In [58]:
from pyspark.ml.evaluation import RegressionEvaluator 

# Evaluate the performance of the model
evaluator1_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
evaluator1_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")

rmse1 = evaluator1_rmse.evaluate(predictions)
r21 = evaluator1_r2.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on test data = {}".format(rmse1))
print("R^2 on test data = {}".format(r21))

25/05/06 09:49:56 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
25/05/06 09:50:11 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


Root Mean Squared Error (RMSE) on test data = 72.84485892524282
R^2 on test data = 0.7276135190670916


In [83]:
metrics_filename = "fine_tuned_linear_regression_metrics"
metrics_data = [{"RMSE": rmse1, "R2": r21, "model": "linear_reg_tuned"}]
metrics_df = spark.createDataFrame(metrics_data)
metrics_df.write.format("json").mode("overwrite").save(f"project/big_data_project/output/{metrics_filename}")
run("hdfs dfs -get project/big_data_project/output/fine_tuned_linear_regression_metrics ../output/model1_tune")

''

# Second model

## Build a model

In [59]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(maxIter=10)
model_gbt = gbt.fit(train_data)

25/05/06 09:50:14 WARN DAGScheduler: Broadcasting large task binary with size 1630.7 KiB
25/05/06 09:50:14 WARN DAGScheduler: Broadcasting large task binary with size 1630.8 KiB
25/05/06 09:50:14 WARN DAGScheduler: Broadcasting large task binary with size 1634.9 KiB
25/05/06 09:50:15 WARN DAGScheduler: Broadcasting large task binary with size 1895.2 KiB
25/05/06 09:50:28 WARN DAGScheduler: Broadcasting large task binary with size 1896.1 KiB
25/05/06 09:50:32 WARN DAGScheduler: Broadcasting large task binary with size 1896.8 KiB
25/05/06 09:50:35 WARN DAGScheduler: Broadcasting large task binary with size 1898.0 KiB
25/05/06 09:50:39 WARN DAGScheduler: Broadcasting large task binary with size 1900.4 KiB
25/05/06 09:50:44 WARN DAGScheduler: Broadcasting large task binary with size 1907.8 KiB
25/05/06 09:50:50 WARN DAGScheduler: Broadcasting large task binary with size 1908.3 KiB
25/05/06 09:50:54 WARN DAGScheduler: Broadcasting large task binary with size 1908.9 KiB
25/05/06 09:50:58 WAR

## Predict for test data

In [60]:
predictions = model_gbt.transform(test_data)
predictions.show()

25/05/06 09:54:24 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


+--------------------+-----+--------------------+-----------------+
|            features|label|     indexedFeatures|       prediction|
+--------------------+-----+--------------------+-----------------+
|(9436,[0,3082,309...| 56.0|(9436,[0,3082,309...|87.58780532556878|
|(9436,[0,3082,309...| 65.0|(9436,[0,3082,309...|89.69886703721815|
|(9436,[0,3082,309...| 99.0|(9436,[0,3082,309...|86.72297685617565|
|(9436,[0,3082,309...| 55.0|(9436,[0,3082,309...|78.27495048004113|
|(9436,[0,3082,309...|175.0|(9436,[0,3082,309...|86.72297685617565|
|(9436,[0,3082,309...| 70.0|(9436,[0,3082,309...|87.58780532556878|
|(9436,[0,3082,309...| 98.0|(9436,[0,3082,309...|88.83403856782502|
|(9436,[0,3082,309...| 80.0|(9436,[0,3082,309...|88.83403856782502|
|(9436,[0,3082,309...| 85.0|(9436,[0,3082,309...|78.07007380111293|
|(9436,[0,3082,309...| 70.0|(9436,[0,3082,309...|88.83403856782502|
|(9436,[0,3082,309...| 85.0|(9436,[0,3082,309...|78.27495048004113|
|(9436,[0,3082,309...| 58.0|(9436,[0,3082,309...

## Evaluate the model

In [61]:
from pyspark.ml.evaluation import RegressionEvaluator 

# Evaluate the performance of the model
evaluator2_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
evaluator2_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")

rmse2 = evaluator2_rmse.evaluate(predictions)
r22 = evaluator2_r2.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on test data = {}".format(rmse2))
print("R^2 on test data = {}".format(r22))

25/05/06 09:54:40 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
25/05/06 09:54:55 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


Root Mean Squared Error (RMSE) on test data = 69.91688790908475
R^2 on test data = 0.7490703886968049


In [84]:
# save metrics 
metrics_filename = "GBT_no_tune"
metrics_data = [{"RMSE": rmse2, "R2": r22, "model": "GBT_no_tune"}]
metrics_df = spark.createDataFrame(metrics_data)
metrics_df.write.format("json").mode("overwrite").save(f"project/big_data_project/output/{metrics_filename}")
run("hdfs dfs -get project/big_data_project/output/GBT_no_tune ../output/model2_no_tune")

''

## Hyperparameter optimization

In [62]:
model_gbt.params

[Param(parent='GBTRegressor_2054749e2086', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'),
 Param(parent='GBTRegressor_2054749e2086', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'),
 Param(parent='GBTRegressor_2054749e2086', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'a

In [63]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator 

import numpy as np

grid = ParamGridBuilder()\
    .addGrid(gbt.maxDepth, [3, 5])\
    .addGrid(gbt.minInstancesPerNode, [1, 2])\
    .build()
cv = CrossValidator(estimator = gbt, 
                    estimatorParamMaps = grid, 
                    evaluator = evaluator2_rmse,
                    parallelism = 5,
                    numFolds=3)

cvModel = cv.fit(train_data)
bestModel = cvModel.bestModel
bestModel

25/05/06 09:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1608.5 KiB
25/05/06 09:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1608.5 KiB
25/05/06 09:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1608.5 KiB
25/05/06 09:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1608.5 KiB
25/05/06 09:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1648.4 KiB
25/05/06 09:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1648.4 KiB
25/05/06 09:54:59 WARN DAGScheduler: Broadcasting large task binary with size 1648.4 KiB
25/05/06 09:55:00 WARN DAGScheduler: Broadcasting large task binary with size 1648.4 KiB
25/05/06 09:55:00 WARN DAGScheduler: Broadcasting large task binary with size 1648.4 KiB
25/05/06 09:55:00 WARN DAGScheduler: Broadcasting large task binary with size 1648.4 KiB
25/05/06 09:55:00 WARN DAGScheduler: Broadcasting large task binary with size 1648.4 KiB
25/05/06 09:55:00 WAR

GBTRegressionModel: uid=GBTRegressor_2054749e2086, numTrees=10, numFeatures=9436

## Best model 2


In [64]:
from pprint import pprint
model2 = bestModel
pprint(model2.extractParamMap())

{Param(parent='GBTRegressor_2054749e2086', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval.'): False,
 Param(parent='GBTRegressor_2054749e2086', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext.'): 10,
 Param(parent='GBTRegressor_2054749e2086', name='maxMemoryInMB', doc='Maximum memory in MB allocated to histogram aggregation. If too small, then 1 node will be split per iteration, and its aggregates may exceed this size.'): 256,
 Param(parent='GBTRegressor_2054749e2086', name='labelCol', doc='label column name.'): 

## Save the model to HDFS

In [65]:
model2.write().overwrite().save("project/big_data_project/models/model2")

# Run it from root directory of the repository
run("hdfs dfs -get project/big_data_project/models/model2 model2")

get: `model2/model1/data/_SUCCESS': File exists                                 
get: `model2/model1/metadata/_SUCCESS': File exists
get: `model2/model1/metadata/part-00000': File exists


''

## Predict for test data using best model2

In [66]:
predictions = model2.transform(test_data)
predictions.show()

25/05/06 10:27:40 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


+--------------------+-----+--------------------+-----------------+
|            features|label|     indexedFeatures|       prediction|
+--------------------+-----+--------------------+-----------------+
|(9436,[0,3082,309...| 56.0|(9436,[0,3082,309...|87.54431936748031|
|(9436,[0,3082,309...| 65.0|(9436,[0,3082,309...|  89.655530427895|
|(9436,[0,3082,309...| 99.0|(9436,[0,3082,309...|87.50828110921341|
|(9436,[0,3082,309...| 55.0|(9436,[0,3082,309...|79.06578543883641|
|(9436,[0,3082,309...|175.0|(9436,[0,3082,309...|87.50828110921341|
|(9436,[0,3082,309...| 70.0|(9436,[0,3082,309...|88.36509874556303|
|(9436,[0,3082,309...| 98.0|(9436,[0,3082,309...| 89.6194921696281|
|(9436,[0,3082,309...| 80.0|(9436,[0,3082,309...| 89.6194921696281|
|(9436,[0,3082,309...| 85.0|(9436,[0,3082,309...|78.84892703881185|
|(9436,[0,3082,309...| 70.0|(9436,[0,3082,309...| 89.6194921696281|
|(9436,[0,3082,309...| 85.0|(9436,[0,3082,309...|79.06578543883641|
|(9436,[0,3082,309...| 58.0|(9436,[0,3082,309...

In [67]:
predictions.select("label", "prediction")\
    .coalesce(1)\
    .write\
    .mode("overwrite")\
    .format("csv")\
    .option("sep", ",")\
    .option("header","true")\
    .save("project/big_data_project/output/model2_predictions.csv")

# Run it from root directory of the repository
run("hdfs dfs -cat project/big_data_project/output/model2_predictions.csv/*.csv > ../output/model2_predictions.csv")

25/05/06 10:27:56 WARN DAGScheduler: Broadcasting large task binary with size 3.3 MiB


''

## Evaluate the best model2

In [88]:
from pyspark.ml.evaluation import RegressionEvaluator 

# Evaluate the performance of the model
evaluator2_rmse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
evaluator2_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")

rmse22 = evaluator2_rmse.evaluate(predictions)
r22 = evaluator2_r2.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on test data = {}".format(rmse2))
print("R^2 on test data = {}".format(r22))

25/05/06 11:17:26 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB
25/05/06 11:17:41 WARN DAGScheduler: Broadcasting large task binary with size 3.1 MiB


Root Mean Squared Error (RMSE) on test data = 69.88522182315187
R^2 on test data = 0.749297634487301


In [89]:
# save metrics 
metrics_filename = "GBT_tune"
metrics_data = [{"RMSE": rmse22, "R2": r22, "model": "GBT_tune"}]
metrics_df = spark.createDataFrame(metrics_data)
metrics_df.write.format("json").mode("overwrite").save(f"project/big_data_project/output/{metrics_filename}")
run("hdfs dfs -get project/big_data_project/output/GBT_tune ../output/model2_tune")

''

# Compare best models

In [90]:
models = [[str(model1),rmse1, r21], [str(model2),rmse2, r22]]

df = spark.createDataFrame(models, ["model", "RMSE", "R2"])
df.show(truncate=False)

+--------------------------------------------------------------------------------+-----------------+------------------+
|model                                                                           |RMSE             |R2                |
+--------------------------------------------------------------------------------+-----------------+------------------+
|LinearRegressionModel: uid=LinearRegression_a9df76969288, numFeatures=9436      |72.84485892524282|0.7276135190670916|
|GBTRegressionModel: uid=GBTRegressor_2054749e2086, numTrees=10, numFeatures=9436|69.88522182315187|0.749297634487301 |
+--------------------------------------------------------------------------------+-----------------+------------------+



In [91]:
df.coalesce(1)\
    .write\
    .mode("overwrite")\
    .format("csv")\
    .option("sep", ",")\
    .option("header","true")\
    .save("project/big_data_project/output/evaluation.csv")
# Run it from root directory of the repository
run("hdfs dfs -cat project/big_data_project/output/evaluation.csv/*.csv > ../output/evaluation.csv")

''

In [92]:
# train_data.count()

In [93]:
print("sth")

sth
